In [37]:
import os
import analysis
import hrf_tools
import numpy as np
from utility import mask_cifti,unmask_cifti
#import matplotlib
#matplotlib.rcParams['figure.figsize'] = (6, 4)
# this is run by run_datalad_run_encoding_model.sh
# "python ./run_encoding_model.py $sub $2"

In [2]:
# 1: extract auditory areas (?)
mmp_59k=np.load('../sourcedata/data/parcellations/mmp_59k.npy' )
early_auditory_59k=np.load('../sourcedata/data/parcellations/early_auditory_59k.npy' )
auditory_59k=np.load('../sourcedata/data/parcellations/auditory_59k.npy' )
#no... these are just the surface and don't match up exactly with the cifti

# run model on just specific features and combine across movies?
# plot results of specific features


# 2 extract primary auditory cortex
# get mfs features? (coarse)
# try to plot tonotopy

In [3]:
subject=100610
feature='as_scores'
dataset='HCP_7T'

#temp_dir = '../../tmp'
#if not os.path.exists(temp_dir):
#    os.mkdir(temp_dir)

#n_movies=[1,2,3,4]

n_movies=[1]
X,Y,vertex_info = analysis.load_data_HCP(subject,feature,n_movies)
Y = np.multiply(Y,auditory_59k)

X = hrf_tools.apply_optimal_hrf_10hz(X,1)
scores_mean,corr_mean,weights_mean = analysis.simple_ridgeCV(X,Y)
#np.save(f'{temp_dir}/r2_{subject}_{feature}.npy',scores_mean)
#np.save(f'{temp_dir}/r_{subject}_{feature}.npy',corr_mean)
#np.save(f'{temp_dir}/w_{subject}_{feature}.npy',weights_mean)
analysis.plot_results(scores_mean,'r2','59k',vertex_info,subject,feature,dataset,'testing')

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
/om2/vast/gablab/jsmentch/projects/nat_img/code/npp.py:12: RuntimeWarning: invalid value encountered in true_divide
  zscore = lambda v: (v-v.mean(0))/v.std(0)
/om2/vast/gablab/jsmentch/projects/nat_img/code/npp.py:12: RuntimeWarning: invalid value encountered in true_divide
  zscore = lambda v: (v-v.mean(0))/v.std(0)
/om2/vast/gablab/jsmentch/projects/nat_img/code/npp.py:12: RuntimeWarning: invalid value encountered in true_divide
  zscore = lambda v: (v-v.mean(0))/v.std(0)
/om2/vast/gablab/jsmentch/projects/nat_img/code/npp.py:12: RuntimeWarning: invalid value encountered in true_divide
  zscore = lambda v: (v-v.mean(0))/v.std(0)
/om2/vast/gablab/jsmentch/projects/nat_img/code/npp.py:12: RuntimeWarning: invalid value encountered in true_divide
  zscore = lambda v: (v-v.mean(0))/v.std(0)


Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data


/om2/vast/gablab/jsmentch/projects/nat_img/code/nilearn_plotting_custom.py:184: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  axes = Axes3D(figure, rect=[0, 0, 1, 1],


[-1.1300591  -1.11564683 -1.0653404  ...  0.28958702  0.29298819
  2.        ]
[-1.28297963 -1.2453177  -1.15000152 ...  0.20792316  0.22223421
  2.        ]
[-1.1300591  -1.11564683 -1.0653404  ...  0.28958702  0.29298819
  2.        ]
[-1.28297963 -1.2453177  -1.15000152 ...  0.20792316  0.22223421
  2.        ]


In [35]:
features=['chroma', 'mfcc', 'mfs', 'as_embed']

subject=100610
dataset='HCP_7T'
#feature=
for feature in features:

    #temp_dir = '../../tmp'
    #if not os.path.exists(temp_dir):
    #    os.mkdir(temp_dir)

    n_movies=[1,2,3,4]

    #n_movies=[1]
    X,Y,vertex_info = analysis.load_data_HCP(subject,feature,n_movies)
    
    Y=mask_cifti(auditory_59k,Y)
    #Y = np.multiply(Y,auditory_59k)

    X = hrf_tools.apply_optimal_hrf_10hz(X,1)
    scores_mean,corr_mean,weights_mean = analysis.simple_ridgeCV(X,Y)
    #np.save(f'{temp_dir}/r2_{subject}_{feature}.npy',scores_mean)
    #np.save(f'{temp_dir}/r_{subject}_{feature}.npy',corr_mean)
    #np.save(f'{temp_dir}/w_{subject}_{feature}.npy',weights_mean)
#     np.save(f'../outputs/encoding_model/HCP_7T/{feature}/ridgeCV_{subject}_{feature}_scores.npy',scores_mean)
#     np.save(f'../outputs/encoding_model/HCP_7T/{feature}/ridgeCV_{subject}_{feature}_corr.npy',corr_mean)
#     np.save(f'../outputs/encoding_model/HCP_7T/{feature}/ridgeCV_{subject}_{feature}_weights.npy',weights_mean)
    scores_mean=unmask_cifti(auditory_59k,scores_mean)

    analysis.plot_results(scores_mean,'r2','59k',vertex_info,subject,feature,dataset,'ridgeCVtest3')

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data


/om2/vast/gablab/jsmentch/projects/nat_img/code/nilearn_plotting_custom.py:184: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  axes = Axes3D(figure, rect=[0, 0, 1, 1],


[-0.09874155 -0.09447936 -0.09176275 ...  0.08007536  0.08030061
  0.08299533]
[-0.12813727 -0.11142669 -0.10869177 ...  0.07974633  0.08074034
  0.09015352]
[-0.09874155 -0.09447936 -0.09176275 ...  0.08007536  0.08030061
  0.08299533]
[-0.12813727 -0.11142669 -0.10869177 ...  0.07974633  0.08074034
  0.09015352]


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


...

[-6.23846697 -6.03544479 -6.02526766 ... -0.19196159 -0.19058153
  0.        ]
[-6.69174977 -6.62573654 -6.49540362 ... -0.12182349 -0.1206131
  0.        ]
[-6.23846697 -6.03544479 -6.02526766 ... -0.19196159 -0.19058153
  0.        ]
[-6.69174977 -6.62573654 -6.49540362 ... -0.12182349 -0.1206131
  0.        ]


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


Cannot load file ../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.sulc_1.6mm_MSMAll.59k_fs_LR.dscalar.nii with sulcal depth data


/om2/vast/gablab/jsmentch/projects/nat_img/code/nilearn_plotting_custom.py:184: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  axes = Axes3D(figure, rect=[0, 0, 1, 1],


[-1.10090557 -1.01409173 -1.01194066 ...  0.21562191  0.21690966
  0.31819095]
[-1.3501126  -1.2752137  -1.24016803 ...  0.34846168  0.37130277
  0.37309082]
[-1.10090557 -1.01409173 -1.01194066 ...  0.21562191  0.21690966
  0.31819095]
[-1.3501126  -1.2752137  -1.24016803 ...  0.34846168  0.37130277
  0.37309082]


## apply the roi better - don't set things to zero, actually extract it

In [16]:
auditory_59k_ind=np.argwhere(auditory_59k==1)[:,1]

array([ 12116,  12117,  12118, ..., 108403, 108404, 108412])

In [17]:
auditory_59k.shape

(1, 170494)

In [18]:
Y.shape

(3162, 170494)

In [21]:
mask_test=mask_cifti(auditory_59k,Y)
mask_test.shape

(3162, 6307)

In [26]:
unmask_test =unmask_cifti(auditory_59k,mask_test)

In [28]:
unmask_test

0.00031051583837644214

In [6]:
auditory_59k.shape
auditory_59k_ind=np.argwhere(auditory_59k==1)

(1, 170494)

In [ ]:
roi_91k_ind = np.squeeze(np.argwhere(roi_91k==1)) #indices of the AC voxels


Ybrain_test_AC = Ybrain_test[:,roi_91k_ind]


In [ ]:
#put them back
back = np.zeros(170494)

r2_91k[roi_91k_ind] = r2[:len(roi_91k_ind)]

(6307,)

In [9]:
auditory_59k.shape

(1, 170494)